In [ ]:
import torch.nn as nn
import torch
import os
import random
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
from numpy.random import rand, randn
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from DogsCatsDataloader import DogsCatsDataset
from torchvision import transforms, utils
from torch.utils.data import DataLoader
from GaborLayers import GaborConv2d

In [ ]:
transform = transforms.Compose([transforms.ToPILImage(), transforms.Resize((256,256)), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
train_set = DogsCatsDataset(root_dir='D:\\Dogs vs Cats\\train', transform=transform)
test_set = DogsCatsDataset(root_dir='D:\\Dogs vs Cats\\validation', transform=transform)

In [ ]:
train = DataLoader(train_set, batch_size=64, shuffle=True, num_workers=4)
test = DataLoader(test_set, batch_size=64, shuffle=True, num_workers=4)

In [ ]:
len(train)

In [ ]:
one_layer_gnet_acc_train = []
one_layer_gnet_loss_train = []
one_layer_gnet_acc_test = []
one_layer_gnet_loss_test = []

two_layer_gnet_acc_train = []
two_layer_gnet_loss_train = []
two_layer_gnet_acc_test = []
two_layer_gnet_loss_test = []

convnet_acc_train = []
convnet_loss_train = []
convnet_acc_test = []
convnet_loss_test = []

one_layer_cgnet_acc_train = []
one_layer_cgnet_loss_train = []
one_layer_cgnet_acc_test = []
one_layer_cgnet_loss_test = []

two_layer_cgnet_acc_train = []
two_layer_cgnet_loss_train = []
two_layer_cgnet_acc_test = []
two_layer_cgnet_loss_test = []

In [ ]:
class GaborNet_one_layer(nn.Module):
    def __init__(self):
        super(GaborNet_one_layer, self).__init__()
        self.g1 = GaborConv2d(3, 32, kernel_size=(15,15), stride=1)
        self.c1 = nn.Conv2d(32, 64, kernel_size=(3,3), stride=1)
        self.c2 = nn.Conv2d(64, 128, kernel_size=(3,3), stride=1)
        self.c3 = nn.Conv2d(128, 128, kernel_size=(3,3), stride=1)
        self.c4 = nn.Conv2d(128, 128, kernel_size=(3,3), stride=1)
        self.fc1 = nn.Linear(128*5*5, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128,2)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.g1(x)), kernel_size=2)
        x = nn.Dropout2d()(x)
        x = F.max_pool2d(F.relu(self.c1(x)), kernel_size=2)
        x = nn.Dropout2d()(x)
        x = F.max_pool2d(F.relu(self.c2(x)), kernel_size=2)
        x = nn.Dropout2d()(x)
        x = F.max_pool2d(F.relu(self.c3(x)), kernel_size=2)
        x = nn.Dropout2d()(x)
        x = F.max_pool2d(F.relu(self.c4(x)), kernel_size=2)
        x = nn.Dropout2d()(x)
        x = x.view(-1, 128*5*5)
        x = F.relu(self.fc1(x))
        x = nn.Dropout()(x)
        x = F.relu(self.fc2(x))
        x = nn.Dropout()(x)
        x = self.fc3(x)
        return x

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.xavier_uniform_(m.weight.data)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
net = GaborNet_one_layer().to(device)
net.apply(weights_init)
print(net)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [ ]:
for epoch in range(100):

    running_loss = 0.0
    correct = 0
    net.train()
    for i, data in enumerate(train, 0):
        # get the inputs
        inputs, labels = data['image'], data['target']

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.to(device))
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()
        pred = outputs.max(1, keepdim=True)[1].to('cpu')
        correct += pred.eq(labels.view_as(pred)).sum().item()

        # print statistics
        running_loss += loss.item()
    print('[%d] train_acc: %.3f train_loss: %.3f' %(epoch + 1, correct/len(train_set), running_loss/len(train_set))) 
    one_layer_gnet_acc_train.append(correct/len(train_set))
    one_layer_gnet_loss_train.append(running_loss/len(train_set)) 
    
    running_loss = 0.0
    correct = 0
    with torch.no_grad():
        net.eval()
        for i, data in enumerate(test, 0):
            # get the inputs
            inputs, labels = data['image'], data['target']

            # forward + backward + optimize
            outputs = net(inputs.to(device))
            loss = criterion(outputs, labels.to(device))

            pred = outputs.max(1, keepdim=True)[1].to('cpu')
            correct += pred.eq(labels.view_as(pred)).sum().item()
            running_loss += loss.item()
    print('[%d] test_acc: %.3f test_loss: %.3f' %(epoch + 1, correct/len(test_set), running_loss/len(test_set)))
    one_layer_gnet_acc_test.append(correct/len(test_set))
    one_layer_gnet_loss_test.append(running_loss/len(test_set))      

print('Finished Training')

np.save('one_layer_gnet_acc_train.npy', one_layer_gnet_acc_train)
np.save('one_layer_gnet_loss_train.npy', one_layer_gnet_loss_train)
np.save('one_layer_gnet_acc_test.npy', one_layer_gnet_acc_test)
np.save('one_layer_gnet_loss_test.npy', one_layer_gnet_loss_test)


'''for epoch in range(100):  # loop over the dataset multiple times
    running_loss = 0.0
    for i in range(int(EMONEW.lenght()/BATCH_SIZE)):
        net.train()
        # get the inputs
        inputs, labels = EMONEW.get_batch(BATCH_SIZE)
        inputs = torch.Tensor(inputs).to(device)/255
        labels = torch.Tensor(labels).max(1)[1].type(torch.LongTensor).to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        # print statistics
        running_loss += loss.item()
        if i%10 == 9:
            print('[%d] loss: %.3f' %(epoch + 1, running_loss / (10*BATCH_SIZE)))
            running_loss = 0.0
            
        optimizer.step()
        
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i in range(int(EMONEW.lenght(False)/BATCH_SIZE)):
            inputs, labels = EMONEW.get_batch(BATCH_SIZE, False)
            inputs = torch.Tensor(inputs).to(device)/255
            outputs = net(inputs)
            labels = torch.Tensor(labels).max(1)[1].type(torch.LongTensor).to(device)
            test_loss += criterion(outputs, labels) # sum up batch loss
            pred = outputs.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(labels.view_as(pred)).sum().item()

    test_loss /= EMONEW.lenght(False)
    print('\n[{}] Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(epoch+1,
        test_loss, correct, EMONEW.lenght(False),
        100. * correct / EMONEW.lenght(False)))
    S_gabor_test_acc.append(correct / EMONEW.lenght(False))
print('Finished Training')'''

In [ ]:
class convNet(nn.Module):
    def __init__(self):
        super(convNet, self).__init__()
        self.c0 = nn.Conv2d(3, 32, kernel_size=(5,5), stride=1)
        self.c1 = nn.Conv2d(32, 64, kernel_size=(5,5), stride=1)
        self.c2 = nn.Conv2d(64, 128, kernel_size=(3,3), stride=1)
        self.c3 = nn.Conv2d(128, 128, kernel_size=(3,3), stride=1)
        self.c4 = nn.Conv2d(128, 128, kernel_size=(3,3), stride=1)
        self.fc1 = nn.Linear(128*5*5, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128,2)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.c0(x)), kernel_size=2)
        x = nn.Dropout2d()(x)
        x = F.max_pool2d(F.relu(self.c1(x)), kernel_size=2)
        x = nn.Dropout2d()(x)
        x = F.max_pool2d(F.relu(self.c2(x)), kernel_size=2)
        x = nn.Dropout2d()(x)
        x = F.max_pool2d(F.relu(self.c3(x)), kernel_size=2)
        x = nn.Dropout2d()(x)
        x = F.max_pool2d(F.relu(self.c4(x)), kernel_size=2)
        x = nn.Dropout2d()(x)
        x = x.view(-1, 128*5*5)
        x = F.relu(self.fc1(x))
        x = nn.Dropout()(x)
        x = F.relu(self.fc2(x))
        x = nn.Dropout()(x)
        x = self.fc3(x)
        return x

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
net = convNet().to(device)
net.apply(weights_init)
print(net)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [ ]:
for epoch in range(100):

    running_loss = 0.0
    correct = 0
    net.train()
    for i, data in enumerate(train, 0):
        # get the inputs
        inputs, labels = data['image'], data['target']

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.to(device))
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()
        pred = outputs.max(1, keepdim=True)[1].to('cpu')
        correct += pred.eq(labels.view_as(pred)).sum().item()

        # print statistics
        running_loss += loss.item()
    print('[%d] train_acc: %.3f train_loss: %.3f' %(epoch + 1, correct/len(train_set), running_loss/len(train_set))) 
    convnet_acc_train.append(correct/len(train_set))
    convnet_loss_train.append(running_loss/len(train_set)) 
    
    running_loss = 0.0
    correct = 0
    with torch.no_grad():
        net.eval()
        for i, data in enumerate(test, 0):
            # get the inputs
            inputs, labels = data['image'], data['target']

            # forward + backward + optimize
            outputs = net(inputs.to(device))
            loss = criterion(outputs, labels.to(device))

            pred = outputs.max(1, keepdim=True)[1].to('cpu')
            correct += pred.eq(labels.view_as(pred)).sum().item()
            running_loss += loss.item()
    print('[%d] test_acc: %.3f test_loss: %.3f' %(epoch + 1, correct/len(test_set), running_loss/len(test_set)))
    convnet_acc_test.append(correct/len(test_set))
    convnet_loss_test.append(running_loss/len(test_set))      

print('Finished Training')

np.save('convnet_acc_train.npy', convnet_acc_train)
np.save('convnet_loss_train.npy', convnet_loss_train)
np.save('convnet_acc_test.npy', convnet_acc_test)
np.save('convnet_loss_test.npy', convnet_loss_test)

In [ ]:
one_layer_gnet_acc_train = np.load('one_layer_gnet_acc_train.npy')

In [ ]:
convnet_acc_train = np.load('convnet_acc_train.npy')
convnet_acc_test = np.load('convnet_acc_test.npy')

In [ ]:
plt.plot(convnet_acc_test, label = 'conv')
plt.plot(one_layer_gnet_acc_test, label = 'Gnet')
plt.legend(loc = 0)
plt.title('test accuracy')
plt.show()
plt.plot(convnet_acc_train, label = 'conv')
plt.plot(one_layer_gnet_acc_train, label = 'Gnet')
plt.legend(loc = 0)
plt.title('train accuracy')
plt.show()

In [ ]:
class GaborNet_two_layer(nn.Module):
    def __init__(self):
        super(GaborNet_two_layer, self).__init__()
        self.g1 = GaborConv2d(3, 32, kernel_size=(15,15), stride=1)
        self.g2 = GaborConv2d(32, 64, kernel_size=(15,15), stride=1)
        self.c2 = nn.Conv2d(64, 128, kernel_size=(3,3), stride=1)
        self.c3 = nn.Conv2d(128, 128, kernel_size=(3,3), stride=1)
        self.c4 = nn.Conv2d(128, 128, kernel_size=(3,3), stride=1)
        self.fc1 = nn.Linear(128*4*4, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128,2)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.g1(x)), kernel_size=2)
        x = nn.Dropout2d()(x)
        x = F.max_pool2d(F.relu(self.g2(x)), kernel_size=2)
        x = nn.Dropout2d()(x)
        x = F.max_pool2d(F.relu(self.c2(x)), kernel_size=2)
        x = nn.Dropout2d()(x)
        x = F.max_pool2d(F.relu(self.c3(x)), kernel_size=2)
        x = nn.Dropout2d()(x)
        x = F.max_pool2d(F.relu(self.c4(x)), kernel_size=2)
        x = nn.Dropout2d()(x)
        print(x.shape)
        x = x.view(-1, 128*4*4)
        x = F.relu(self.fc1(x))
        x = nn.Dropout()(x)
        x = F.relu(self.fc2(x))
        x = nn.Dropout()(x)
        x = self.fc3(x)
        return x

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
net = GaborNet_two_layer().to(device)
net.apply(weights_init)
print(net)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [ ]:
net(torch.normal(torch.Tensor(1,3,256,256)).to(device))